In [18]:
import pandas as pd
import numpy as np
import math
import yaml

pd.options.display.max_columns = 9999
pd.set_option("display.float_format", "${:,.0f}".format)

In [19]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


# Input

In [20]:
MAX_SUPPLY = 1_500_000_000

In [21]:
with open("input_saage.yaml", "r") as stream:
    distribution = yaml.safe_load(stream)

In [22]:
CONSTANTS = {"total_months": 48}

# Calculation

In [23]:
categories = distribution.keys()

df = pd.DataFrame(columns=categories, index=range(CONSTANTS["total_months"])).fillna(0)

In [26]:
for month in range(CONSTANTS["total_months"]):
    for category in categories:
        params = distribution[category]

        if month == 0 and "initial_supply" in params:
            df.loc[month, category] = params["initial_supply"]
        else:

            if not params.get("exponential", False):
                if (
                    params["month_offset"]
                    <= month
                    < params["month_offset"] + params["month_distribution"]
                ):
                    df.loc[month, category] = (
                        params["weight"] * MAX_SUPPLY - params.get("initial_supply", 0)
                    ) / max(1, params["month_distribution"])
            else:
                if month > 0:
                    # find period

                    period = 0
                    start_period = 0
                    month_id = -1

                    for i, month_period in enumerate(
                        params["exponential_arg"]["month"]
                    ):

                        if month <= start_period + month_period:

                            month_id = i
                            month_len = params["exponential_arg"]["month"][i]
                            weight = params["exponential_arg"]["weight"][i]
                            break
                        else:
                            start_period += month_period

                    if month_id >= 0 :

                        current_scalar = math.exp(-month / 12)
                        scalars = [
                            math.exp(-m / 12)
                            for m in range(
                                start_period + 1, start_period + month_period + 1
                            )
                        ]

                        distribution_amount = params["weight"] * MAX_SUPPLY * weight


                        df.loc[month, category] = (
                    current_scalar / sum(scalars) * distribution_amount
                        )

0.9200444146293233 6.0714483406218624 25500000.000000004
3864173.959296961
0.8464817248906141 6.0714483406218624 25500000.000000004
3555211.668407247
0.7788007830714049 6.0714483406218624 25500000.000000004
3270952.6383430855
0.7165313105737893 6.0714483406218624 25500000.000000004
3009421.705424604
0.6592406302004438 6.0714483406218624 25500000.000000004
2768801.63134016
0.6065306597126334 6.0714483406218624 25500000.000000004
2547420.4761310727
0.5580351457700471 6.0714483406218624 25500000.000000004
2343739.980776765
0.513417119032592 6.0714483406218624 25500000.000000004
2156344.8786571
0.4723665527410147 6.0714483406218624 25500000.000000004
1983933.06162301
0.4345982085070782 3.8781998233819026 178500000.0
20003038.458927352
0.39984965434484737 3.8781998233819026 178500000.0
18403683.809751656
0.36787944117144233 3.8781998233819026 178500000.0
16932206.497766115
0.3384654251067422 3.8781998233819026 178500000.0
15578382.01562005
0.3114032239145977 3.8781998233819026 178500000.0
1

In [25]:
df["total_free_supply"] = df.sum(axis=1).cumsum()

In [80]:
df["total_free_supply_percent"] = df.total_free_supply / MAX_SUPPLY

In [81]:
df["max_circulating_supply"] = df.total_free_supply - df.loc[0, "INSURANCE FUND"]

In [89]:
df.sum(axis=0)

BETTING INCENTIVES                  $25,000,000
BETTING REBATES                     $25,000,000
IDO                                 $30,000,000
INSURANCE FUND                     $100,000,000
LP PROVIDERS                       $200,000,000
SEED                                $50,000,000
STAKERS                            $170,000,000
STAKING AIRDROP                     $20,000,000
TEAM                               $180,000,000
TREASURY                           $200,000,000
total_free_supply           $41,183,906,265,131
total_free_supply_percent               $41,184
max_circulating_supply      $41,179,106,265,131
dtype: float64

In [82]:
df

,BETTING INCENTIVES,BETTING REBATES,IDO,INSURANCE FUND,LP PROVIDERS,SEED,STAKERS,STAKING AIRDROP,TEAM,TREASURY,total_free_supply,total_free_supply_percent,max_circulating_supply
0,$0,$0,30000000,100000000,$0,50000000,$0,20000000,0,"$50,000,000","$1,450,000,001",$1,"$1,350,000,001"
1,$0,$0,0,0,"$23,060,158",0,"$1,393,034",0,5000000,"$4,166,667","$3,713,706,474",$4,"$3,613,706,474"
2,$0,$0,0,0,"$22,508,094",0,"$2,018,746",0,5000000,"$4,166,667","$7,157,280,661",$7,"$7,057,280,661"
3,$0,$0,0,0,"$21,631,747",0,"$3,440,604",0,5000000,"$4,166,667","$11,926,410,022",$12,"$11,826,410,022"
4,$0,$0,0,0,"$20,240,633",0,"$9,774,795",0,5000000,"$4,166,667","$18,163,693,535",$18,"$18,063,693,535"
5,$0,$0,0,0,"$18,032,377",0,"$14,088,533",0,5000000,"$4,166,667","$25,993,505,179",$26,"$25,893,505,179"
6,$0,$0,0,0,"$14,526,989",0,"$11,305,958",0,5000000,"$4,166,667","$35,517,397,481",$36,"$35,417,397,481"
7,$0,$0,0,0,"$4,509,358",0,"$4,252,960",0,5000000,"$4,166,667","$46,780,630,703",$47,"$46,680,630,703"
8,$0,$0,0,0,"$4,502,975",0,"$2,702,091",0,5000000,"$4,166,667","$59,873,753,286",$60,"$59,773,753,286"
9,$0,$0,0,0,"$4,496,385",0,"$2,023,280",0,5000000,"$4,166,667","$74,871,395,831",$75,"$74,771,395,831"


In [83]:
rates = [90, 70, 50, 30]

In [84]:
output_yields = pd.DataFrame(columns=["type", "rate", "year", "percent"])

In [85]:
print("Staking yield\n---------------")
for rate in rates:
    print(f"Rate {rate}%")
    year_one_rate = df.loc[1:12, "STAKERS"].sum() / (
        df.loc[0, "max_circulating_supply"] * rate / 100
    )
    year_two_rate = df.loc[13:24, "STAKERS"].sum() / (
        df.loc[12, "max_circulating_supply"] * rate / 100
    )
    year_three_rate = df.loc[25:36, "STAKERS"].sum() / (
        df.loc[24, "max_circulating_supply"] * rate / 100
    )

    print("\tYear one rate:", "{:.2%}".format(year_one_rate))
    print("\tYear two rate:", "{:.2%}".format(year_two_rate))
    print("\tYear three rate:", "{:.2%}".format(year_three_rate))

    output_yields.loc[len(output_yields) + 1] = [
        "staking yields",
        rate,
        1,
        year_one_rate,
    ]
    output_yields.loc[len(output_yields) + 1] = [
        "staking yields",
        rate,
        2,
        year_two_rate,
    ]
    output_yields.loc[len(output_yields) + 1] = [
        "staking yields",
        rate,
        3,
        year_three_rate,
    ]

Staking yield
---------------
Rate 90%
	Year one rate: 4.50%
	Year two rate: 0.07%
	Year three rate: 0.01%
Rate 70%
	Year one rate: 5.79%
	Year two rate: 0.09%
	Year three rate: 0.01%
Rate 50%
	Year one rate: 8.10%
	Year two rate: 0.12%
	Year three rate: 0.01%
Rate 30%
	Year one rate: 13.51%
	Year two rate: 0.21%
	Year three rate: 0.02%


In [86]:
print("LP yield\n---------------")
for rate in rates:
    print(f"Rate {rate}%")
    year_one_rate = df.loc[1:12, "LP PROVIDERS"].sum() / (
        df.max_circulating_supply.loc[0] * (1 - rate / 100)
    )
    year_two_rate = df.loc[13:24, "LP PROVIDERS"].sum() / (
        df.max_circulating_supply.loc[12] * (1 - rate / 100)
    )
    year_three_rate = df.loc[25:36, "LP PROVIDERS"].sum() / (
        df.max_circulating_supply.loc[24] * (1 - rate / 100)
    )

    print("\tYear one rate:", "{:.2%}".format(year_one_rate))
    print("\tYear two rate:", "{:.2%}".format(year_two_rate))
    print("\tYear three rate:", "{:.2%}".format(year_three_rate))

    output_yields.loc[len(output_yields) + 1] = ["lp yields", rate, 1, year_one_rate]
    output_yields.loc[len(output_yields) + 1] = ["lp yields", rate, 2, year_two_rate]
    output_yields.loc[len(output_yields) + 1] = ["lp yields", rate, 3, year_three_rate]

LP yield
---------------
Rate 90%
	Year one rate: 108.86%
	Year two rate: 0.40%
	Year three rate: 0.00%
Rate 70%
	Year one rate: 36.29%
	Year two rate: 0.13%
	Year three rate: 0.00%
Rate 50%
	Year one rate: 21.77%
	Year two rate: 0.08%
	Year three rate: 0.00%
Rate 30%
	Year one rate: 15.55%
	Year two rate: 0.06%
	Year three rate: 0.00%


In [87]:
output_yields

,type,rate,year,percent
1,staking yields,90,1,$0
2,staking yields,90,2,$0
3,staking yields,90,3,$0
4,staking yields,70,1,$0
5,staking yields,70,2,$0
6,staking yields,70,3,$0
7,staking yields,50,1,$0
8,staking yields,50,2,$0
9,staking yields,50,3,$0
10,staking yields,30,1,$0


In [50]:
path = r"output.xlsx"

writer = pd.ExcelWriter(path, engine="xlsxwriter")
df.to_excel(writer, sheet_name="table")
output_yields.to_excel(writer, sheet_name="yields")
writer.save()
writer.close()